# Extraction of PS order parameter from the `.mpack` files of the saved models

In [15]:
import netket as nk
import numpy as np
import time
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import jax
import flax
import optax
from sys import version as pyvers

print("Python version: {}".format(pyvers))
print("NetKet version: {}".format(nk.__version__))
print("NumPy version: {}".format(np.__version__))

Python version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
NetKet version: 3.3.2.post2
NumPy version: 1.20.3


In [16]:
"""lattice"""	
SITES    = 64            # 4, 8, 16, 20 ... number of vertices in a tile determines the tile shape 	
JEXCH1   = .2            # nn interaction	
JEXCH2   = 1            # nnn interaction	
H_Z      = 0             # magnetic field
USE_PBC = True
TOTAL_SZ = None            # 0, None ... restriction of Hilbert space	
"""machine learning"""	
SAMPLER = 'local'       # 'local' = MetropolisLocal, 'exchange' = MetropolisExchange, 'exact' = ExactSampler
MACHINE = 'RBM'         # 'RBM', 'RBMSymm'
DTYPE = np.complex128   # type of weights in neural network
ALPHA = 8              # N_hidden / N_visible	
ETA   = .01            # learning rate (0.01 usually works)	
SAMPLES = 3000	
NUM_ITER = 1000
N_PRE_ITER = 100        # number of iteration before checking for convergence to speed up the process if the model is already pre-trained
STEPS = np.arange(0.0,1.21,step=0.1)

In [17]:
from lattice_and_ops import Lattice
lattice = Lattice(SITES)

if not USE_PBC and SITES != 16:
    raise Exception("Non-PBC are implemented only for 4x4 lattice!!!")

# Construction of custom graph according to tiled lattice structure defined in the Lattice class.
edge_colors = []
for node in range(SITES):
    if USE_PBC or not node in [3,7,11,15]:
        edge_colors.append([node,lattice.rt(node), 1])  # horizontal connections
    if USE_PBC or not node in [12,13,14,15]:
        edge_colors.append([node,lattice.bot(node), 1]) # vertical connections
    row, column = lattice.position(node)
    
    SS_color = 3 if not USE_PBC and node in [3,7,4,12,13,14,15] else 2
    if column%2 == 0:
        if row%2 == 0:
            edge_colors.append([node,lattice.lrt(node),SS_color]) # diagonal bond
        else:
            edge_colors.append([node,lattice.llft(node),SS_color]) # diagonal bond

g = nk.graph.Graph(edges=edge_colors) #,n_nodes=3)
N = g.n_nodes

hilbert = nk.hilbert.Spin(s=.5, N=g.n_nodes, total_sz=TOTAL_SZ)

In [18]:
from lattice_and_ops import HamOps
ho = HamOps()
# H_Z = .5
ha_1 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=False), bond_ops_colors=ho.bond_color)
ha_2 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=True), bond_ops_colors=ho.bond_color)


In [19]:
from lattice_and_ops import Operators
ops = Operators(lattice,hilbert,ho.mszsz,ho.exchange)

In [20]:
machine_1 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)
machine_2 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)
sampler_1 = nk.sampler.MetropolisLocal(hilbert=hilbert)
sampler_2 = nk.sampler.MetropolisLocal(hilbert=hilbert)
vs_1 = nk.vqs.MCState(sampler_1 , machine_1 , n_samples=SAMPLES)
vs_2 = nk.vqs.MCState(sampler_2 , machine_2 , n_samples=SAMPLES)

/home/mezic/.local/lib/python3.8/site-packages/netket/vqs/mc/mc_state/state.py:58: UserWarning: n_samples=3000 (3000 per MPI rank) does not divide n_chains=16, increased to 3008 (3008 per MPI rank)
  warnings.warn(


In [21]:
import netket.experimental

In [24]:
ops.m_sSquared_slow_MSR(vs_2) 

(DeviceArray(0.00219133+2.59585949e-05j, dtype=complex128),
 DeviceArray(2802.56073217, dtype=float64))

In [23]:
vs_1.expect(ops.m_plaquette_op)

0.054+0.000j ± 0.018 [σ²=0.194, R̂=1.3976]

In [25]:
for i in STEPS:
    loaded_variables_1 = nk.experimental.vqs.variables_from_file("resurrection/RBM64_"+str(i)+"_0.mpack",vs_1.variables)
    loaded_variables_2 = nk.experimental.vqs.variables_from_file("resurrection/RBM64_"+str(i)+"_1.mpack",vs_2.variables)
    vs_1.variables = loaded_variables_1
    vs_2.variables = loaded_variables_2
    # the evaluation m_sSquared_slow_MSR for 8x8 lattice takes 6 min on my CPU
    print(i,vs_1.expect(ops.m_plaquette_op),vs_2.expect(ops.m_plaquette_op_MSR),ops.m_sSquared_slow_MSR(vs_2))

In [ ]:
vs_1.estimate(ops.m_plaquette_op)

AttributeError: 'MCState' object has no attribute 'estimate'

In [ ]:
print(vs_1.expect(ops.m_plaquette_op),vs_2.expect(ops.m_plaquette_op_MSR),ops.m_sSquared_slow_MSR(vs_2))

-0.0201-0.0093j ± 0.0083 [σ²=0.1968, R̂=0.9997] -0.062+0.000j ± 0.026 [σ²=1.991, R̂=1.0008] (DeviceArray(1.0295763-2.37662473e-06j, dtype=complex128), DeviceArray(8182.94793354, dtype=float64))


In [ ]:
vs_1.variables

FrozenDict({
    params: {
        Dense: {
            bias: array([ 6.48484625e-04-4.07984931e-03j, -4.90526479e-03+8.77478696e-03j,
                   -4.29097056e-04-2.59454449e-04j,  9.43851840e-03-1.69783355e-04j,
                    4.89548174e-03-3.80195064e-03j, -1.00666884e-03+1.34071519e-02j,
                    1.70427333e-04-4.22334720e-03j, -1.45193873e-02-8.31194728e-03j,
                   -1.92879375e-03+5.63607998e-03j, -6.01019814e-03+6.99569384e-03j,
                    9.44238614e-03-2.87684757e-03j, -9.05078882e-03+5.58516766e-04j,
                   -1.88721761e-03-2.58142642e-03j,  9.25154095e-04+1.65252382e-02j,
                   -6.32560499e-03+1.02220456e-02j,  2.67755027e-03-1.48809542e-02j,
                    4.82341522e-03+4.04232024e-03j, -2.27735530e-03-1.05569236e-02j,
                   -5.62300236e-03+1.35476248e-02j, -1.25080267e-02+1.63741168e-02j,
                   -7.60359968e-03-6.76140569e-03j, -1.22274154e-02-5.58603484e-03j,
               

In [ ]:
vs_1.parameters

FrozenDict({
    Dense: {
        bias: DeviceArray([ 9.39966074e-03-1.05939241e-03j,
                     -3.55306603e-03-2.22497191e-03j,
                     -1.06583727e-02-2.69274145e-03j,
                     -4.84892762e-04+5.33893103e-03j,
                      8.71762833e-03+7.02434290e-03j,
                     -1.08561894e-02+4.50367391e-03j,
                     -7.54858193e-03+5.96972535e-03j,
                     -1.34719831e-03-9.95093553e-03j,
                      1.01100034e-02+1.91453048e-03j,
                      1.20308273e-02+2.47166372e-03j,
                     -2.51862546e-03+9.27869927e-03j,
                     -5.20217758e-03-9.77629304e-03j,
                     -7.63720433e-03+1.39534250e-03j,
                     -3.87673527e-03-1.08455261e-02j,
                      5.87201479e-03-2.60812485e-04j,
                      6.39883583e-03-9.88157906e-03j,
                      3.28503135e-03-7.65201178e-03j,
                     -1.76560581e-03+2.83811399e-0